<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/tensorflow_sensitivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sensitivity of Option Price to Implied Volatility Surface


The sensitivity of the option price $ C(K, T) $ to the implied volatility surface c \sigma_{\text{implied}}(K, T) $ is computed as:

$
\frac{\partial C(K, T)}{\partial \sigma_{\text{implied}}(K, T)}.
$

This measures how a small change in the implied volatility at a specific strike $ K $ and maturity $ T $ affects the option price.


### Black-Scholes Option Pricing Formula

The Black-Scholes option pricing formula for a European call option is given by:

$
C(K, T) = S_0 \Phi(d_1) - K e^{-rT} \Phi(d_2),
$

where:

$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{1}{2} \sigma^2\right) T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T}.
$

**Parameters:**
- $ S_0 $: Spot price of the underlying asset,
- $ K $: Strike price of the option,
- $ T $: Time to maturity,
- $ r $: Risk-free rate,
- $ \sigma $: Volatility,
- $ \Phi(x) $: Cumulative distribution function (CDF) of the standard normal distribution.


### Volatility Surface Representation

The implied volatility surface is represented as a grid of strike prices $ K $ and maturities $ T $:

$
\sigma_{\text{implied}}(K, T) =
\begin{bmatrix}
\sigma_{1,1} & \sigma_{1,2} & \cdots & \sigma_{1,n} \\
\sigma_{2,1} & \sigma_{2,2} & \cdots & \sigma_{2,n} \\
\vdots       & \vdots       & \ddots & \vdots       \\
\sigma_{m,1} & \sigma_{m,2} & \cdots & \sigma_{m,n} \\
\end{bmatrix}.
$


### Sensitivity Calculation

The sensitivity of the option price to the implied volatility surface is calculated by summing the option prices over the grid and taking the gradient with respect to the volatility surface:

$
\frac{\partial C}{\partial \sigma_{\text{implied}}} =
\begin{bmatrix}
\frac{\partial C_{1,1}}{\partial \sigma_{1,1}} & \frac{\partial C_{1,2}}{\partial \sigma_{1,2}} & \cdots & \frac{\partial C_{1,n}}{\partial \sigma_{1,n}} \\
\frac{\partial C_{2,1}}{\partial \sigma_{2,1}} & \frac{\partial C_{2,2}}{\partial \sigma_{2,2}} & \cdots & \frac{\partial C_{2,n}}{\partial \sigma_{2,n}} \\
\vdots                                        & \vdots                                        & \ddots & \vdots                                        \\
\frac{\partial C_{m,1}}{\partial \sigma_{m,1}} & \frac{\partial C_{m,2}}{\partial \sigma_{m,2}} & \cdots & \frac{\partial C_{m,n}}{\partial \sigma_{m,n}} \\
\end{bmatrix}.
$


#### **6. Markdown Cell: Output Interpretation**
### Output Interpretation

The output of the sensitivity calculation is a matrix of the same size as the implied volatility surface. Each entry represents:

$
\text{sensitivity}_{ij} = \frac{\partial C(K_i, T_j)}{\partial \sigma_{\text{implied}}(K_i, T_j)}.
$

This matrix quantifies the effect of changes in implied volatility on option prices across the entire surface.


### Applications

The sensitivity of option prices to the volatility surface is useful for:

- Quantifying the impact of changes in implied volatility on option prices.
- Developing risk management and hedging strategies.
- Calibrating local volatility models for more accurate pricing.


In [ ]:
import tensorflow as tf

# Parameters
r = 0.03
T = tf.constant([0.1, 0.5, 1.0, 2.0], dtype=tf.float32)
K = tf.constant([50, 60, 70, 80, 90, 100, 110, 120, 130], dtype=tf.float32)
implied_vol_surface = tf.Variable(
    [
        [0.20, 0.19, 0.18, 0.17, 0.16, 0.15, 0.16, 0.17, 0.18],
        [0.21, 0.20, 0.19, 0.18, 0.17, 0.16, 0.17, 0.18, 0.19],
        [0.22, 0.21, 0.20, 0.19, 0.18, 0.17, 0.18, 0.19, 0.20],
        [0.23, 0.22, 0.21, 0.20, 0.19, 0.18, 0.19, 0.20, 0.21],
    ],
    dtype=tf.float32,
)
S0 = tf.constant(100.0, dtype=tf.float32)

# Option price function
def option_price(S0, K, T, sigma, r):
    d1 = (tf.math.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * tf.sqrt(T))
    d2 = d1 - sigma * tf.sqrt(T)
    N = tf.compat.v1.distributions.Normal(0.0, 1.0)
    return S0 * N.cdf(d1) - K * tf.exp(-r * T) * N.cdf(d2)

@tf.function
def compute_option_prices(vol_surface):
    T_expanded = tf.expand_dims(T, axis=1)
    K_expanded = tf.expand_dims(K, axis=0)
    return option_price(S0, K_expanded, T_expanded, vol_surface, r)

In [ ]:
# Autograd sensitivity
with tf.GradientTape() as tape:
    tape.watch(implied_vol_surface)
    option_prices = compute_option_prices(implied_vol_surface)
autograd_sensitivity = tape.gradient(option_prices, implied_vol_surface)

# Finite difference method
delta = 1e-5
finite_diff_sensitivity = tf.zeros_like(implied_vol_surface)
for i in range(implied_vol_surface.shape[0]):
    for j in range(implied_vol_surface.shape[1]):
        vol_plus = tf.Variable(implied_vol_surface) # Use implied_vol_surface directly
        vol_minus = tf.Variable(implied_vol_surface) # Use implied_vol_surface directly
        vol_plus[i, j].assign(vol_plus[i, j] + delta) # Use assign and + operator
        vol_minus[i, j].assign(vol_minus[i, j] - delta) # Use assign and - operator
        C_plus = compute_option_prices(vol_plus)
        C_minus = compute_option_prices(vol_minus)
        sensitivity = (tf.reduce_sum(C_plus) - tf.reduce_sum(C_minus)) / (2 * delta)
        finite_diff_sensitivity = tf.tensor_scatter_nd_update(
            finite_diff_sensitivity, [[i, j]], [sensitivity]
        )

In [ ]:
# prompt: print without scientific notation
import numpy as np
print("Autograd Sensitivity:\n", np.array_str(autograd_sensitivity.numpy(), precision=4, suppress_small=True))
print("\nFinite Difference Sensitivity:\n", np.array_str(finite_diff_sensitivity.numpy(), precision=4, suppress_small=True))

In [ ]:
# Compare results
relative_error = tf.norm(finite_diff_sensitivity - autograd_sensitivity) / tf.norm(autograd_sensitivity)
print(f"Relative Error: {relative_error.numpy()}")